### Задачи к Лекции 4

__Исходные данные__ 

Дан файл **"mlbootcamp5_train.csv"**. В нем содержатся данные об опросе 70000 пациентов с целью определения наличия заболеваний сердечно-сосудистой системы (ССЗ). Данные в файле промаркированы и если у человека имееются ССЗ, то значение **cardio** будет равно 1, в противном случае - 0. Описание и значения полей представлены во второй лекции.

__Загрузка файла__

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from math import *
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]


df = pd.read_csv("../data/mlbootcamp5_train.csv", 
                 sep=";", 
                 index_col="id")
df  = df[(df['ap_hi'] >= 50) & (df['ap_hi'] <= 220) & (df['ap_lo'] >= 50)\
         & (df['ap_lo'] <= 175) & (df['height'] >= 100) & (df['height'] <= 250)\
         & (df['weight'] >= 30) & (df['weight'] <= 220)]
# Делаем one-hot кодирование
chol = pd.get_dummies(df["cholesterol"], prefix="chol")
gluc = pd.get_dummies(df["gluc"], prefix="gluc")
df = pd.concat([df, chol, gluc], axis=1)

# Делаем пол бинарным признаком
df["gender_bin"] = df["gender"].map({1: 0, 2: 1})
df.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,chol_1,chol_2,chol_3,gluc_1,gluc_2,gluc_3,gender_bin
id,,,,,,,,,,,,,,,,,,,
0,18393,2,168,62.0,110,80,1,1,0,0,1,0,1,0,0,1,0,0,1
1,20228,1,156,85.0,140,90,3,1,0,0,1,1,0,0,1,1,0,0,0
2,18857,1,165,64.0,130,70,3,1,0,0,0,1,0,0,1,1,0,0,0
3,17623,2,169,82.0,150,100,1,1,0,0,1,1,1,0,0,1,0,0,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0,1,0,0,1,0,0,0


## Задачи

__1. Хоть в sklearn и присутствует реализация метода k-ближайших соседей, я же предлагаю попробовать вам написать его самостоятельно.__

* __создать классификатор используя только pandas, numpy и scipy. Гиперпараметром данного классификатора должно быть число ближайших соседей. (Необязательно) можно добавить метрику расстояния и выбор весов.__
* __С помощью кросс-валидации найти оптимальное количество ближайших соседей и (необязательно) набор признаков.__

Алгоритм работы классификатора:
 1. Для заданного прецедент  $\vec{x}$ мы считаем расстояние до всех прецедентов в обучающей выборке.
 2. Сортируем прецеденты по расстоянию до $\vec{x}$.
 3. Отбираем $k$ минимальных значений
 4. Устраиваем голосование между отобранными прецедентами.

In [25]:
from sklearn.model_selection import train_test_split
from collections import Counter


# Нормализация по миниМаксу 
def df_MinMax(df):
    df_MinMax = df.apply(lambda x: (x-min(x))/(max(x)-min(x)))
    return df_MinMax

#Вычисление расстояния между двумя векторами
def distance(a_test, b_train, k):
    a_ar = a_test
    b_ar = b_train 
    dist = np.ndarray([a_test.shape[0], k], dtype = int)
    for m, value in enumerate(a_ar):
        dist[m] = np.argsort(np.sqrt(np.sum((b_ar - value)**2, axis = 1)))[:k]
    return dist   
#        for column in a.columns:
#            if column != 'cardio':
#                absum += ((a[column].iloc[i] - b[column].iloc[i])**2)
#        dist.append(np.sqrt(absum))
#    return dist 
    

# Классификатор по методу k - наименьших соседей
class KNN:
    
    def __init__(self, k=3):
        self.k = k
    
    def fit(self, X, y):
        self.X_train = X.to_numpy()
        self.y_train = y.to_numpy()
    
    def predict(self, X):
        #pred_labels = []
        pred_labels = [self._predict(x) for x in X.to_numpy()]
        return np.array(pred_labels)
        
    
    def _predict(self, x):
        #knndist = []
        #knlabels = []
        #knndist[distance(x, self.X_train) for i in range(x.shape[0])]
        #k_inc = np.argsort(knndist)[:self.k]
        #knlabels.append(self.y_train.iloc[i] for i in k_inc)
        k_inc = distance(x, self.X_train, self.k)[0]
        knlabels = self.y_train.take(k_inc)
        most_comm = Counter(knlabels).most_common(1)
        return most_comm[0][0]
    
    
    
    
  


dfmm = df_MinMax(df)
target = dfmm['cardio']
train, test, target_train, target_test = train_test_split(dfmm, target, test_size = 0.30) 
#
#
#    
clf = KNN(k = 3)
clf.fit(train,target_train)
preds = clf.predict(test)
#        
print(preds)
#print(distance(test, train))
#test.shape[0]
#acc = np.sum(preds == target_test) / target_test.shape[0]    
    
##df_min_max = kek[0].drop(['cardio'],axis=1)
#df_min_max = kek[0]
#target_min_max = kek[0]['cardio']
##df_min_max.iloc[1][0]
##df_min_max.iloc[1].shape[0]
#train, test, target_train, target_test = train_test_split(df_min_max, target_min_max, test_size = 0.30) 
#
################################################################################
##Main classification procedure
#def classifyKNN (train, test, k, numberOfClasses):
#    #Euclidean distance between 2-dimensional point
#    def dist (a, b):
#        ab_sum = 0
#        for i in range(a.shape[0]):
#            ab_sum += (a.iloc[i] - b.iloc[i])**2
#            distance = sqrt(ab_sum)
#        return distance
##################################################################################    
#    #Claculate distances between test point and all of the train points
#    train_no_cardio = train.loc[:, df.columns != 'cardio']
#    test_no_cardio = test.loc[:, df.columns != 'cardio']
#    
#    results = []
#    #df_dist = pd.DataFrame([],columns=["test_id", "Distance", "cardio"])
#    #df_empty = df_dist
#    
#    for i in range(train_no_cardio.shape[0]):
#        #df_dist = df_empty;
#        for j in range(test_no_cardio.shape[0]):
#            
#            dist_test = dist(train_no_cardio.iloc[i],test_no_cardio.iloc[j])
#            results.append((j,dist_test,test.iloc[j]['cardio']))
#        df_dist = pd.DataFrame(results, columns=["test_id", "distance", "cardio"])
#            
#
#                
#        
#        if i == 2: 
#                break
#        
#        #train.iloc[i][]
#        #dist_test = dist(trainPoint,test.iloc[i]) 
#        #dist_test.
#    return df_dist.sort_values(by="distance", ascending = True).head(k)#dist_test
#
##dr = pd.DataFrame(results, columns=["Name", "F1", "Accuracy", "ROC AUC"])
##dr['Log_Loss'] = pd.Series(ll_list, index=dr.index)
#
#test1 = classifyKNN(train,test,9,2)
#test1
#
##df.loc[:, df.columns != 'cardio']
##train
#
##test
#
##def calculateAccuracy (nClasses, nItemsInClass, k, testPercent):
#    #train, test, target_train, target_test = train_test_split(df_min_max, target_min_max, test_size = testPercent) 
#
##calculateAccuracy(2,df_min_max.shape(0),10,0.3)    
#
#
##dist_test = dist(df_min_max.iloc[1],df_min_max.iloc[2])
##dist_test



[1. 1. 1. ... 1. 0. 0.]


**Комментарии:** Ваши комментарии здесь.

**2. Определить какой из трех классификаторов (kNN, наивный Байес, решающее дерево) лучший в каждой метрике по отдельности: accuracy, F1-мера, ROC AUC, функция потерь. Использовать набор признаков: 'age', 'weight', 'height', 'ap_lo', 'ap_hi'.**

**(Необязательно) Найти оптимальный набор признаков.**

In [84]:
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss


data = df[['age', 'weight', 'height', 'ap_lo', 'ap_hi']]
target_comp = df['cardio']


classifiers = [("Tree (depth=%d)" % i, DecisionTreeClassifier(max_depth=i, random_state=13)) for i in range(1, 15)]
classifiers.append(("GaussianNB()", GaussianNB()))
for i in range(1, 15):
    classifiers.append(("kNN (neighbors=%d)" % i, KNeighborsClassifier(n_neighbors=i))) 

results = []
ll_list = []
for name, classifier in classifiers:
    cv = cross_validate(classifier, data, target_comp, n_jobs=-1, cv=5, scoring=['accuracy', 'f1', 'roc_auc'])
    results.append((name, cv["test_f1"].mean(), cv["test_accuracy"].mean(), cv["test_roc_auc"].mean()))
    model = classifier.fit(data, target_comp)
    predict = model.predict_proba(data)[:,1]
    ll_list.append(log_loss(target_comp, predict, eps = 1e-15, normalize = True))
    
        
dr = pd.DataFrame(results, columns=["Name", "F1", "Accuracy", "ROC AUC"])
dr['Log_Loss'] = pd.Series(ll_list, index=dr.index)
print("ROC_AUC\n", dr.sort_values(by="ROC AUC")[-1:])
print()
print("Accuracy\n", dr.sort_values(by="Accuracy")[-1:])
print()
print("F1\n", dr.sort_values(by="F1")[-1:])
print()
print("LogLoss\n", dr.sort_values(by="Log_Loss")[-1:])
print()
dr


ROC_AUC
              Name        F1  Accuracy   ROC AUC  Log_Loss
5  Tree (depth=6)  0.708471  0.719814  0.783876  0.553011

Accuracy
              Name       F1  Accuracy   ROC AUC  Log_Loss
3  Tree (depth=4)  0.70909  0.719887  0.780242  0.559101

F1
              Name       F1  Accuracy   ROC AUC  Log_Loss
3  Tree (depth=4)  0.70909  0.719887  0.780242  0.559101

LogLoss
             Name        F1  Accuracy   ROC AUC  Log_Loss
14  GaussianNB()  0.666821  0.708651  0.777948  0.642009



,Name,F1,Accuracy,ROC AUC,Log_Loss
0,Tree (depth=1),0.679722,0.712566,0.711583,0.596465
1,Tree (depth=2),0.679722,0.712566,0.763546,0.569333
2,Tree (depth=3),0.708017,0.719872,0.775553,0.562619
3,Tree (depth=4),0.709090,0.719887,0.780242,0.559101
4,Tree (depth=5),0.706500,0.719829,0.782818,0.555758
5,Tree (depth=6),0.708471,0.719814,0.783876,0.553011
6,Tree (depth=7),0.704539,0.718737,0.782622,0.549594
7,Tree (depth=8),0.706366,0.719727,0.781042,0.545263
8,Tree (depth=9),0.702093,0.718737,0.777764,0.539450
9,Tree (depth=10),0.699723,0.716161,0.771927,0.531504


**Комментарии:** Ваши комментарии здесь.